In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pytorch_lightning==1.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [ ]:
Q_TKN = '<usr>'
A_TKN = '<sys>'
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [ ]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
Chatbot_Data = pd.read_csv('/content/drive/MyDrive/T3Q-자연어/CHATBOT/chatbot_dataset.csv', encoding='cp949')

FileNotFoundError: ignored

In [ ]:
# Chatbot_Data.head()

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# encoder = LabelEncoder()
# Chatbot_Data['label'] = encoder.fit_transform(Chatbot_Data['label'].values)

In [ ]:
# Chatbot_Data.head()

In [ ]:
# Chatbot_Data.info()

In [ ]:
import math
import numpy as np
import pandas as pd
import random
import re
import torch
import logging
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast

In [ ]:
# class ChatbotDataset(Dataset):
#     def __init__(self, chats, max_len=32):
#         self._data = chats
#         self.first = True
#         self.q_token = Q_TKN
#         self.a_token = A_TKN
#         self.sent_token = SENT
# #         self.bos = BOS
#         self.eos = EOS
#         self.mask = MASK
# #         self.pad = PAD
#         self.max_len = max_len
#         self.tokenizer = koGPT2_TOKENIZER 

#     def __len__(self):
#         return len(self._data)

#     def __getitem__(self, idx):
#         turn = self._data.iloc[idx]
#         q = turn['Q']
#         a = turn['A']
#         sentiment = str(turn['label'])
#         q_toked = self.tokenizer.tokenize(self.q_token + q + \
#                                           self.sent_token + sentiment)   
#         q_len = len(q_toked)
#         a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
#         a_len = len(a_toked)
#         if q_len + a_len > self.max_len:
#             a_len = self.max_len - q_len
#             if a_len <= 0:
#                 q_toked = q_toked[-(int(self.max_len/2)):]
#                 q_len = len(q_toked)
#                 a_len = self.max_len - q_len
#                 assert a_len > 0
#             a_toked = a_toked[:a_len]
#             a_len = len(a_toked)
#             assert a_len == len(a_toked), f'{a_len} ==? {len(a_toked)}'
#         # [mask, mask, ...., mask, ..., <bos>,..A.. <eos>, <pad>....]
#         labels = [
#             self.mask,
#         ] * q_len + a_toked[1:]
#         if self.first:
#             logging.info("contexts : {}".format(q))
#             logging.info("toked ctx: {}".format(q_toked))
#             logging.info("response : {}".format(a))
#             logging.info("toked response : {}".format(a_toked))
#             logging.info('labels {}'.format(labels))
#             self.first = False
#         mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
#         self.max_len
#         labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
#         while len(labels_ids) < self.max_len:
#             labels_ids += [self.tokenizer.pad_token_id]
#         token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
#         while len(token_ids) < self.max_len:
#             token_ids += [self.tokenizer.pad_token_id]
#         return(token_ids, np.array(mask),
#                labels_ids)

In [ ]:
# def collate_batch(batch):
#     data = [item[0] for item in batch]
#     mask = [item[1] for item in batch]
#     label = [item[2] for item in batch]
#     return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# train_set = ChatbotDataset(Chatbot_Data, max_len=40)
# train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [ ]:
# model.to(device)
# model.train()

NameError: ignored

In [ ]:
# model = torch.load('/content/drive/MyDrive/T3Q-자연어/CHATBOT/model_saved/model_save_v10.pt', map_location = device)

In [ ]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)
epoch = 2
Sneg = -1e18

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
# print('start')
# for epoch in range(epoch):
#     for batch_idx, samples in enumerate(tqdm(train_dataloader)):
#         optimizer.zero_grad()
#         token_ids, mask, label = samples
#         out = model(token_ids.to(device))
#         out = out.logits.to(device)
#         mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2).to(device)
#         mask_out = torch.where(mask_3d==1, out, Sneg+torch.ones_like(out).to(device)).to(device)
#         loss = criterion(mask_out.transpose(2, 1), label.to(device))
#         avg_loss = loss.sum() / mask.sum()
#         avg_loss.backward()
#         optimizer.step()
#     scheduler.step()  
#     torch.save(model, f'/content/drive/MyDrive/T3Q-자연어/CHATBOT/model_saved/model_save_v{epoch + 9}.pt')
#     print(f'save_epoch: {epoch}')
# print('end')

In [ ]:
model = torch.load('/content/drive/MyDrive/T3Q-자연어/CHATBOT/model_saved/model_save_v10.pt', map_location = device)

In [ ]:
# sent = '268'
with torch.no_grad():
    while 1:
        q = input('user > ').strip()
        if q == 'quit':
            break
        a = ''
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + A_TKN + a)).unsqueeze(dim=0).to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred.to('cpu'), dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace('▁', " ")
        print("Chatbot > {}".format(a.strip()))

user > 사는게 너무 힘들다
Chatbot > 가끔은 내려놓기도 하죠.
user > 안녕
Chatbot > 걱정할 상황은 아니신 것 같아요. 가끔 상황과 분위기에 따라 더 많이 먹게 될 때도 있잖아요. 걱정 마세요.
user > 너무 행복해
Chatbot > 더 행복해질 거예요.
user > quit
